In [2]:
import numpy as np
import math
import random

In [37]:
# Start with an N which is a product of primes.
N=15
b=13#random.randint(2,N-2)
print("b value: ",b)
# Check that there is no common divisor.
g=math.gcd(b,N)
if g>1:
    print("We found the factors from gcd(b,N)")
    print("Factors: ",g,int(round(N/g)))
else:
    n=math.ceil(2*math.log(N)/math.log(2))
    Q=2**n
    print("Q value:", Q)
    x=random.randint(0,Q-1)
    # Find a value of f(x) at random.  This simulates measurement of the |f(x)> register.
    d=pow(b,x,N)
    print("f(x) value:", d)
    # Create a vector with equal weightings for all values of x which give that value for f(x).
    v=np.zeros((Q,))
    s=0
    pw=1
    for x in range(Q):
        pw=(pw*b)%N
        if pw==d:
            v[x]=1
            s=s+1
    # Normalise it.  Here s kept track of how many values were needed, so we just divide by the square root of s.
    v=v/np.sqrt(s)
    # The fft gives the same result as the quantum Fourier transform, except the resulting state will not be normalised.
    ft=np.fft.fft(v)
    # We take the absolute value squared of amplitudes to give probabilities, then divide by the dimension Q to normalise it.
    ft=abs(ft)**2/Q
    # Next we find the cumulative probability distribution.
    fc=np.zeros((Q,))
    for x in range(1,Q):
        fc[x]=ft[x-1]+fc[x-1]
    fct=0
    tries=0   
    # It is possible that do not get a reasonable result on the first attempt, because the probability of getting
    # results away from the peaks is nonzero.  We use "tries" "to keep track of the number of attempts, and do not try
    # more than 10 times.  The variable fct gives a factor when successful, but is initialised to zero.  This means we
    # just need to test whether fct>0 to know if we have been successful.
    while fct==0 and tries<10:
        tries=tries+1
        # Generate random number to find measurement result.
        rnd=random.random()
        # Find measurement result. The reason why it uses powers of 2 is that it is a more efficient
        # method with complexity n instead of 2^n given the cumulative probability distribution. 
        y=0
        for cnt in range(n-1,-1,-1):
            stp=2**cnt
            if rnd>fc[y+stp]:
                y=y+stp
        print("Measurement result:", y)
        # Next we do continued fractions to find an approximation of y/Q. We first compute the a and xi vectors.
        # The initialisation uses [0]*n for an integer vector and np.zeros((n,)) for a vector of reals.
        # A possible value of y we could get that is not useful is y=0, because that would lead to an estimate of r=0.
        # We therefore first test that y>0.
        if y>0:
            a=[0]*n
            xi=np.zeros((n,))
            a[0]=int(y/Q)
            xi[0]=y/Q-a[0]
            for cnt in range(1,n):
                if xi[cnt-1]>1e-10:
                    a[cnt]=int(1/xi[cnt-1])
                    xi[cnt]=1/xi[cnt-1]-a[cnt]
            # Next we compute the p and q vectors.
            p=[0]*n
            q=[0]*n
            p[0]=a[0]
            q[0]=1
            p[1]=int(round(a[1]*a[0]+1))
            q[1]=a[1]
            # After computing the p and q vectors, we run through the ratios p[k]/q[k] until we find one that is close
            # enough to y/Q.  Since it is possible that p[k] and q[k] could have a common factor, we divide by their
            # greatest common denominator to estimate r.  We first test k=1.
            r=0
            if (abs(y/Q-p[1]/q[1])*Q<1/2):
                gc=math.gcd(p[1],q[1])
                r=int(q[1]/gc)
                print("p and q", p[1]/gc, q[1]/gc)
            # Next we run through the remainder of the vectors.  At each stage we only update r if r is equal to zero.
            # This ensures that we give the value of r corresponding to the smallest pair of p and q.
            for cnt in range(2,n):
                p[cnt]=a[cnt]*p[cnt-1]+p[cnt-2]
                q[cnt]=a[cnt]*q[cnt-1]+q[cnt-2]
                if (r==0) and (abs(y/Q-p[cnt]/q[cnt])<1/(2*Q)):
                    gc=math.gcd(p[cnt],q[cnt])
                    r=int(q[cnt]/gc)
                    print("p and q", p[cnt]/gc, q[cnt]/gc)
            print("Period:", r)
            tmp=pow(b,int(r/2),N)
            # We check that r is not odd, and b^(r/2)+1 mod N is nonzero.
            if (r%2!=1)&((tmp+1)%N !=0):
                f1=math.gcd(tmp-1,N)                
                f2=math.gcd(tmp+1,N)
                print(f1,f2)
                # One of these should be a factor of N. To work out which one we find the two remainders.
                r1=N % f1
                r2=N % f2
                if (r2 == 0) and (f2>1) and (f2<N):
                    fct=f2
                if (r1 == 0) and (f1>1) and (f1<N):
                    fct=f1
                # If fct>0 it means we found a factor, so we print the two factors.
                if fct>0:
                    print("Factors:", fct,int(round(N/fct)))

b value:  13
Q value: 256
f(x) value: 13
Measurement result: 192
p and q 3.0 4.0
Period: 4
3 5
Factors: 3 5
